In [1]:
#libraries:

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import urllib.request
import pandas as pd

In [2]:
#Initialising Lists
all_links = []
all_pages = []

In [3]:
%%time
#Scraping Pages
link = 'https://pk.sapphireonline.pk/collections/western-wear'
all_pages.append(link)
base_link = 'https://pk.sapphireonline.pk'
while (True):
        uClient = uReq(link)
        all_web = uClient.read()
        uClient.close()
        page_soup = soup(all_web, "html.parser")
        pages = page_soup.find('div', {'class':'infinite-scrolling'})
        if (pages == None):
            break
        link = base_link + pages.a['href']
        all_pages.append(link)

all_pages
        

Wall time: 7.59 s


['https://pk.sapphireonline.pk/collections/western-wear',
 'https://pk.sapphireonline.pk/collections/western-wear?page=2',
 'https://pk.sapphireonline.pk/collections/western-wear?page=3']

In [4]:
%%time
#Scraping All Product Links
product_container = []
for link in all_pages:
    uClient = uReq(link)
    all_web = uClient.read()
    uClient.close()
    page_soup = soup(all_web, "html.parser")
    for product in page_soup.findAll("div", {"class":"grid-item col-6 col-md-4 col-xl-3 four-columns"}):
        product_container.append(product)
    #Retrieving All Product Links
    for link in range(len(product_container)):
        x = base_link + product_container[link].a['href']
        all_links.append(x)

all_rows = []
len(all_links)

Wall time: 4.32 s


254

In [5]:
%%time
#Scraping All Products


h = 0
x = 0
faultylinks = []
for link in all_links:
    h += 1
    if (h%5 == 0):    
        print('working...' + str(x))
    
    try:
        uClient = uReq(link)
        all_web = uClient.read()
        uClient.close()
        name = None
        des1 = None
        des2 = None
        detail = None
        color = None
        fabric = None
        oldprice = None
        newprice = None
        all_sizes = None
        extra_detail = None
        product_soup = soup(all_web, "html.parser")
        name = product_soup.find("h2", {"itemprop":"name"}).text.strip()
        content = product_soup.find("section", {"id":'content1'})('p')
        name = product_soup.find("h2", {"itemprop":"name"}).text.strip()
        if (len(content) != 1 and len(content) <= 5):
            if (content[1].text != 'Ready To Wear'):
                if (content[0].strong != None):
                    des1 = content[0].strong.text
                else:
                    b = content[0].text.find("\n")
                    des1 = content[0].text[:b].strip()

                des2 = content[0].text.replace(des1, "").strip()
                details = content[1].text
                z = details.find("Color")
                detail = details[8:z].strip()
                l = details.find("Fabric")
                color = details[z+6:l].strip()
                fabric = details[l+7:].strip()
                if len(content) == 3:
                    extra_detail = content[2].text
            else:
                des1 = content[2].text.strip()
                des2 = content[0].text.strip()
                detail = content[1].text.strip()
                z = content[3].text.find("Fabric")
                color = content[3].text[:z].replace("Color:", "").strip()
                fabric = content[3].text[z:].replace("Fabric:", "").strip()
        elif (len(content) >= 6):
            des1 = content[3].text.strip()
            des2 = content[1].text.strip()
            detail = content[4].text.strip()
            z = content[5].text.find("Fabric")
            color = content[5].text[:z].replace("Color:", "").strip()
            fabric = content[5].text[z:].replace("Fabric:", "").strip()

        else:
            content = content[0]
            des1 = content.strong.text.strip()
            content = content.text
            z = content.find("Details:")
            l = content.find("Color")
            k = content.find("Fabric")
            j = content.find("SIZE")
            des2 = content[len(des1):z].strip()
            detail = content[z:l].replace("Details:", "").strip()
            color = content[l:k].replace("Color:", "").strip()
            fabric = content[k:j].replace("Fabric:", "").strip()


        prices = product_soup.find("div",{"class":"prices"})('span')
        oldprice = prices[1].text.strip()
        newprice = prices[2].text.strip()
        sizes = product_soup.find_all("div", {'class':'swatch-element'})
        all_sizes = []
        types = []
        for size in sizes:
            c = size.label.text.strip()
            if (c == 'XXS' or c == 'XS' or c == 'S' or c == 'M' or c == 'L' or c == 'XL' or c == 'XXL' or c == range(0,50)):
                all_sizes.append(c)
            else:
                types.append(c)

        all_sizes = ','.join(all_sizes)
        types = ','.join(types)
        #Images
        #images = product_soup.find_all("a", {"class":"fancybox"})
        #y = 0
        #for a in images:
            #file_name = str(x) + "." + str(y) + ".jpg"
            #print(a['href'] + file_name)
           # y += 1
            #urllib.request.urlretrieve("https:" + a['href'], file_name)
        
        x += 1
        row = {'Name':name,'Detail':detail,'Main-Description': des1,'Sub-Description':des2, 'Old-Price': oldprice, 'New-Price': newprice,'Color': color, 'Fabric' : fabric, 'Types': types, 'All Sizes' : all_sizes,"Extra Detail" : extra_detail}
        all_rows.append(row)
    except:
        print(link)
        faultylinks.append(link)
        pass

working...4
working...9
working...14
working...19
working...24
working...29
working...34
working...39
working...44
working...49
working...54
working...59
working...64
working...69
working...74
working...79
working...84
working...89
working...94
working...99
working...104
working...109
working...114
working...119
working...124
working...129
working...134
working...139
working...144
working...149
working...154
working...159
working...164
working...169
working...174
working...179
working...184
working...189
working...194
working...199
working...204
working...209
working...214
working...219
working...224
working...229
working...234
working...239
working...244
working...249
Wall time: 7min 11s


In [9]:
products = pd.DataFrame(all_rows, columns = ['Name','Detail','Main-Description', 'Sub-Description','Old-Price','New-Price','Color', 'Fabric','Types','All Sizes',"Extra Detail"])
products

,Name,Detail,Main-Description,Sub-Description,Old-Price,New-Price,Color,Fabric,Types,All Sizes,Extra Detail
0,Reeve,,Jacket,This minimalist kimono style jacket comes in t...,"Rs.7,990.00","Rs.5,193.50",Heather Grey,Melange,,"XS,S,M,L,XL",None
1,Slate,,Jacket,Elegance and minimalism come together in this ...,"Rs.7,990.00","Rs.5,193.50",Navy,Melange,,"XS,S,M,L,XL",None
2,Suki,,Jacket,A kimono style short jacket in a statement bur...,"Rs.7,990.00","Rs.5,193.50",Maroon,Melange,,"XS,S,M,L,XL",None
3,Cerulean,,Jacket,A round collared jacket with utilitarian pocke...,"Rs.7,990.00","Rs.5,193.50",Navy,Melange,,"XS,S,M,L,XL",None
4,BluJay,,Jacket,A classic blazer is an essential building bloc...,"Rs.7,990.00","Rs.5,193.50",Navy,Melange,,"XS,S,M,L,XL",None
5,Tabitha,,Jacket,A shawl collar adds a feminine touch to this m...,"Rs.7,990.00","Rs.5,193.50",Heather Grey,Melange,,"XS,S,M,L,XL",None
6,Wild West,,Boot Cut Pants,This pair of jeans comes in a boot cut and a f...,"Rs.2,290.00","Rs.1,488.50",Navy,Denim,"26,28,30,32,34",,None
7,Stronger,,T-Shirt,A white cotton tee printed with 'Stay Original.',"Rs.1,290.00",Rs.838.50,White,100%cotton,,"XS,S,M,L,XL",None
8,Twilight,,T-Shirt,A soft navy-blue tee in 100% cotton.,"Rs.1,490.00",Rs.968.50,Navy,100%cotton,,"XS,S,M,L,XL",None
9,Air,,Sweat Shirt,An athletic dove grey sweatshirt with short sl...,"Rs.1,290.00",Rs.838.50,Grey,100% Cotton,,"XS,S,M,L,XL",None


In [8]:
products.to_csv('Sapphire-WesternWear.csv', encoding='utf-8')